# Spatiotemporal CNN-RNN Proxy Model for CO2 Monitoring
### Misael M. Morales, The University of Texas at Austin
Morales, M.M., Torres-Verdin, C., and Pyrcz, M. J. (2024) Stochastic pix2vid: A new spatiotemporal deep learning method for image-to-video synthesis in geologic CO2 storage prediction
***

In [ ]:
from utils import *
check_tensorflow_gpu()

proxy = SpatiotemporalCO2()
proxy.__dict__

In [ ]:
proxy.load_data()
proxy.process_data()

In [ ]:
proxy.plot_data()
proxy.plot_features()
proxy.plot_targets()

In [ ]:
proxy.make_model()

In [ ]:
proxy.training()

In [ ]:
proxy.predictions()

In [ ]:
proxy.plot_single_results(411, 'train')

In [ ]:
proxy.cumulative_co2()

***
# END